## SQL

Since sqlite3 is a third-party package it must be imported before doing anything else

In [1]:
import sqlite3

# Connecting to a new database 

If there is no database, it will be created.

A cursor object is required to speak SQL with the database.

The cursor object is used to execute SQL queries on a database. 

You can get a cursor object by calling db_connection.cursor()

In [6]:
db = sqlite3.connect('database.db') # connects to the file database.db and opens it as a database
cur = db.cursor()

In [7]:
print(dir(db)) # prints all connect object methods

['DataError', 'DatabaseError', 'Error', 'IntegrityError', 'InterfaceError', 'InternalError', 'NotSupportedError', 'OperationalError', 'ProgrammingError', 'Warning', '__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'backup', 'close', 'commit', 'create_aggregate', 'create_collation', 'create_function', 'cursor', 'enable_load_extension', 'execute', 'executemany', 'executescript', 'in_transaction', 'interrupt', 'isolation_level', 'iterdump', 'load_extension', 'rollback', 'row_factory', 'set_authorizer', 'set_progress_handler', 'set_trace_callback', 'text_factory', 'total_changes']


# Creating a new table in our database

Syntax:

CREATE TABLE table (
    column1 datatype,
    column2 datatype)


To execute SQL queries on cursors call the execute() or executemany() method

Every table has to have a PRIMARY KEY. 

Another common optional parameter is NOT NULL


In [15]:
cur.execute(''' CREATE TABLE IF NOT EXISTS store(store_id integer PRIMARY KEY, name text, location text )''')

When we make changes to the database they are not saved until we call the commit() method on the connection object

In [17]:
db.commit()

# Create

Syntax:

INSERT INTO table (attribute1, attribute2) VALUES (value1, value2)

Attributes are only necessary to set if you are omitting any attribute.

In [18]:
cur.execute('''INSERT INTO store VALUES(1234, 'Espresso house', 'Trelleborg')''')
cur.execute('''INSERT INTO store VALUES(321, 'Espresso house', 'Lund')''')
db.commit()

In [19]:
data = (71273821, 'Ica', 'Lund')
cur.execute('''INSERT INTO store VALUES (?,?,?)''', data)
db.commit()

# Read

In SQL the read method is called SELECT. 

Syntax: 

SELECT columns FROM table WHERE condition

We need to use a fetch command with the cursor to actually get the data

In [20]:
cur.execute("SELECT name FROM store WHERE location = 'Lund'")
cur.fetchall()

[('Espresso house',), ('Ica',), ('Espresso house',), ('Ica',)]

In [21]:
out = cur.execute('SELECT * FROM store')
for item in out:
    print(item)

(1234, 'Espresso house', 'Trelleborg')
(321, 'Espresso house', 'Lund')
(71273821, 'Ica', 'Lund')
(1234, 'Espresso house', 'Trelleborg')
(321, 'Espresso house', 'Lund')
(71273821, 'Ica', 'Lund')


# Update

To change the attributes of existing tables and rows in a table we can use the UPDATE statement

Syntax: 

UPDATE table SET column1 = x, column2 = y WHERE condition

In [22]:
cur.execute("""UPDATE store 
            SET name = 'EH' 
            WHERE name = 'Espresso house'""")
db.commit()

# Delete

To remove a row from a table we can use the DELETE FROM statement

Syntax: 

DELETE FROM table WHERE condition

To delete a table we can use the statement: DROP TABLE [IF EXISTS] table


In [23]:
cur.execute('DELETE FROM store WHERE store_id = 123')
db.commit()

# Implement a database for stores and their sales

Specs:
- table stores
- table sales
- table products

In [24]:
db = sqlite3.connect('company.db')
cur = db.cursor()

In [25]:
cur.execute('''CREATE TABLE IF NOT EXISTS stores(
    id integer,
    name text,
    location text)''')

cur.execute(""" CREATE TABLE IF NOT EXISTS sales(
    id integer,
    product_id integer,
    store_id integer,
    price float)""")

cur.execute(""" CREATE TABLE IF NOT EXISTS products(
    id integer,
    name text,
    price float)""")

db.commit()

In [26]:
cur.execute("INSERT INTO stores VALUES (123, 'Ica', 'Malmö')")
cur.execute("INSERT INTO stores VALUES (124, 'Ica', 'Lund')")
db.commit()

In [27]:
data = (125, 'Ica', 'Trelleborg')
cur.execute("INSERT INTO stores VALUES (?,?,?)", data)
db.commit()

In [28]:
cur.execute("INSERT INTO products VALUES(1,'Mjölk', 587)")
db.commit()

In [29]:
cur.execute("UPDATE products SET price = 12 WHERE name = 'Mjölk'")
db.commit()

In [30]:
items = cur.execute("SELECT location FROM stores WHERE name = 'Ica' ")
for item in items:
    print(item)

('Malmö',)
('Lund',)
('Trelleborg',)


In [31]:
cur.execute('INSERT INTO sales VALUES(19879, 1, 124, 12)')
cur.execute('INSERT INTO sales VALUES(19880, 1, 123, 12)')
cur.execute('INSERT INTO sales VALUES(19885, 1, 123, 12)')
cur.execute('INSERT INTO sales VALUES(19886, 1, 123, 12)')

db.commit()

## TestDB

In [32]:
import pandas as pd

In [40]:
db = sqlite3.connect('testdb.db')
cur = db.cursor()

## Column constraints


- PRIMARY KEY = NOT NULL + UNIQUE
- NOT NULL
- UNIQUE
- FOREIGN KEY

In [41]:
cur.execute("""CREATE TABLE IF NOT EXISTS store(
id integer PRIMARY KEY,
name text) """)


cur.execute("""CREATE TABLE IF NOT EXISTS sales(
id integer PRIMARY KEY,
store_id integer,
FOREIGN KEY (store_id) REFERENCES store (id))""")



In [42]:
cur.execute("""INSERT INTO store VALUES(2, 'Ica') """)
db.commit()

## Pandas

Vi kan använda pandas för att ladda in data från en relationsdatabas med SQL. Detta kan vara nödvändigt när man måste koppla upp sig till en stor databas där mäbngden data är för stor för att ha i datorns RAM-minne.

Vi börjar med att spara ner en dataframe till en databas som vi sedan kan använda för att testa detta

In [44]:
weather = pd.read_csv('austin_weather.csv')
weather.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,...,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,...,30.13,29.87,10,10,5,16,6,25,0,
2,2013-12-23,58,45,32,31,27,23,76,52,27,...,30.49,30.41,10,10,10,8,3,12,0,
3,2013-12-24,61,46,31,36,28,21,89,56,22,...,30.45,30.3,10,10,7,12,4,20,0,
4,2013-12-25,58,50,41,44,40,36,86,71,56,...,30.33,30.27,10,10,7,10,2,16,T,


In [45]:
db = sqlite3.connect('austin_weather.db')
weather.to_sql('weather', db) #weather blir nament på tabellen i datbasen

## Nu kan vi läsa in data till en dataframe från databasen

syntax: df = pd.read_sql(sql_anrop, db_connection)

In [46]:
df = pd.read_sql("SELECT Date, TempHighF, TempLowF FROM weather", db)
df.head()

,Date,TempHighF,TempLowF
0,2013-12-21,74,45
1,2013-12-22,56,39
2,2013-12-23,58,32
3,2013-12-24,61,31
4,2013-12-25,58,41


In [47]:
df = pd.read_sql("SELECT * FROM weather", db)
df.head()

,index,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,0,2013-12-21,74,60,45,67,49,43,93,75,...,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,1,2013-12-22,56,48,39,43,36,28,93,68,...,30.13,29.87,10,10,5,16,6,25,0,
2,2,2013-12-23,58,45,32,31,27,23,76,52,...,30.49,30.41,10,10,10,8,3,12,0,
3,3,2013-12-24,61,46,31,36,28,21,89,56,...,30.45,30.3,10,10,7,12,4,20,0,
4,4,2013-12-25,58,50,41,44,40,36,86,71,...,30.33,30.27,10,10,7,10,2,16,T,


## Vi vill dra ut alla rader med WindHighMPH > 10 från databasen

In [48]:
df = pd.read_sql("SELECT Date, WindHighMPH, TempHighF FROM weather WHERE WindHighMPH > 10", db)
df.head()

,Date,WindHighMPH,TempHighF
0,2013-12-21,20,74
1,2013-12-22,16,56
2,2013-12-23,8,58
3,2013-12-24,12,61
4,2013-12-26,12,57


#### Det funkade inte. Vi undersöker varför...

In [49]:
df.dtypes

Date           object
WindHighMPH    object
TempHighF       int64
dtype: object

kolumnen WindHighMPH är av typen object och därför har "WHERE WindHighMPH > 10" inte fungerat. För att få ut datan vi vill ha måste vi casta kolumnen till int... 

I vår kolumn hade vi minst ett värde som var '-' och kunde därför inte casta hela kolumnen direkt. Det finns två alternativ för att komma runt detta

#### Alt 1 - boolean masking som specifikt kollar på '-'

In [50]:
df = df[df['WindHighMPH'] != '-' ]


#### Alt 2 - boolean masking som använder apply och en lambda funktion för att kolla om kolumnvärden är digits

In [51]:
df2 = df[ df['WindHighMPH'].apply(lambda x: str(x).isdigit())]

## Casta kolumnen till int

In [52]:
df['WindHighMPH'] = df['WindHighMPH'].astype(int)

In [53]:
df.dtypes

Date           object
WindHighMPH     int64
TempHighF       int64
dtype: object

## nu kan vi använda pandas för att få ut de dagar som hade WindHighMPH > 10

In [54]:
print(len(df)) # Sanity check
subset_df = df[ df['WindHighMPH'] > 10]
len(subset_df)

1177


977

## Bonus! Visualization med altair

Altair är ett bibliotek jag nyligen upptäckt för visualisering som verkar väldigt kraftfullt utan mycket customization

Prova att installera det och kolla in docs på https://altair-viz.github.io/getting_started/starting.html#the-data

In [55]:
!pip install altair

### En enkel lineplot med altair

In [56]:
import altair as alt

data = df[df.Date > '2017-01-01']

alt.Chart(data).mark_line().encode(
    x = 'Date',
    y='TempHighF')

alt.Chart(...)

### Genom att byta ut mark_line mot mark_point får vi en scatterplot istället

In [57]:
alt.Chart(data).mark_point().encode(
    x = 'Date',
    y='TempHighF')

alt.Chart(...)